# README

Kode pada notebook ini kurang lebih sama dengan kode pada [notebook model LightGBM](https://github.com/alifrachmat2002/datathon-ristek/blob/754418e8905aef373d6120f685f3f2c05bba2010/Datathon%20-%20LightGBM.ipynb), silahkan cek untuk keterangan lebih lanjut mengenai kode yang ada pada link tersebut

The code on this notebook is more or less the same as the code on [LightGBM model notebook](https://github.com/alifrachmat2002/datathon-ristek/blob/754418e8905aef373d6120f685f3f2c05bba2010/Datathon%20-%20LightGBM.ipynb), please check for further information about the code in the link

In [4]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

preprocessed_train_data_dir = os.path.join('/kaggle/input/preprocessed-training-data/train_data_preprocessed.csv')
informasi_jalan_dir = os.path.join('/kaggle/input/informasi-jalan/informasi-jalan.csv')

def remove_timezone(dt_str):
    return dt_str.split('+')[0]

train_data = pd.read_csv(preprocessed_train_data_dir, 
                         parse_dates=['waktu_setempat'],
                         converters={'waktu_setempat': remove_timezone})

test_data = pd.read_csv('/kaggle/input/ristek-datathon-2023/test.csv', parse_dates=['waktu_setempat'])

train_data

,waktu_setempat,id_jalan,id_titik_mulai,id_titik_akhir,rerata_kecepatan,rute
0,2020-02-01 00:00:00,691007296,21390008,1425033102,29.126,21390008-1425033102
1,2020-02-01 01:00:00,691007296,21390008,1425033102,29.126,21390008-1425033102
2,2020-02-01 02:00:00,691007296,21390008,1425033102,34.772,21390008-1425033102
3,2020-02-01 03:00:00,691007296,21390008,1425033102,34.161,21390008-1425033102
4,2020-02-01 04:00:00,691007296,21390008,1425033102,39.383,21390008-1425033102
...,...,...,...,...,...,...
493147,2020-02-22 19:00:00,24704286,130234354,1578999260,31.028,130234354-1578999260
493148,2020-02-22 20:00:00,24704286,130234354,1578999260,32.845,130234354-1578999260
493149,2020-02-22 21:00:00,24704286,130234354,1578999260,32.845,130234354-1578999260
493150,2020-02-22 22:00:00,24704286,130234354,1578999260,35.673,130234354-1578999260


In [5]:
info_jalan = pd.read_csv(informasi_jalan_dir)
info_jalan

,w_id,geo,highway,lit,maxspeed,name,ref
0,1210,"LINESTRING (-0.1802294 51.6398113, -0.1801711 ...",primary,yes,30 mph,High Road,A1000
1,3688059,"LINESTRING (-0.2438268 51.4652563, -0.2438951 ...",trunk,yes,30 mph,Roehampton Lane,A306
2,3691841,"LINESTRING (-0.2287468 51.4632189, -0.2287007 ...",trunk,yes,30 mph,Upper Richmond Road,A205
3,4004732,"LINESTRING (-0.2512982 51.4652257, -0.2518281 ...",trunk,yes,30 mph,Upper Richmond Road West,A205
4,4060819,"LINESTRING (-0.1247121 51.459605, -0.1241664 5...",primary,yes,20 mph,Acre Lane,A2217
5,4061596,"LINESTRING (-0.130106 51.4525672, -0.1301601 5...",secondary,yes,20 mph,Kings Avenue,B221
6,4898588,"LINESTRING (-0.1132653 51.4543707, -0.1131892 ...",primary,yes,20 mph,Tulse Hill,A204
7,8504977,"LINESTRING (-0.2368584 51.5322482, -0.236752 5...",primary,yes,30 mph,Harrow Road,A404
8,22932408,"LINESTRING (-0.1634248 51.4483532, -0.1632963 ...",secondary,yes,20 mph,Nightingale Lane,B237
9,24704286,"LINESTRING (0.0203705 51.5457204, 0.0211241 51...",primary,yes,30 mph,Romford Road,A118


In [6]:
info_jalan = info_jalan.rename(columns={'w_id':'id_jalan'})
info_jalan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id_jalan  20 non-null     int64 
 1   geo       20 non-null     object
 2   highway   20 non-null     object
 3   lit       20 non-null     object
 4   maxspeed  20 non-null     object
 5   name      20 non-null     object
 6   ref       20 non-null     object
dtypes: int64(1), object(6)
memory usage: 1.2+ KB


In [7]:
train_data_w_info = train_data.merge(info_jalan,on=['id_jalan'],how='left')
train_data_w_info

,waktu_setempat,id_jalan,id_titik_mulai,id_titik_akhir,rerata_kecepatan,rute,geo,highway,lit,maxspeed,name,ref
0,2020-02-01 00:00:00,691007296,21390008,1425033102,29.126,21390008-1425033102,"LINESTRING (-0.1604803 51.4352873, -0.1601615 ...",trunk,yes,30 mph,Upper Tooting Road,A24
1,2020-02-01 01:00:00,691007296,21390008,1425033102,29.126,21390008-1425033102,"LINESTRING (-0.1604803 51.4352873, -0.1601615 ...",trunk,yes,30 mph,Upper Tooting Road,A24
2,2020-02-01 02:00:00,691007296,21390008,1425033102,34.772,21390008-1425033102,"LINESTRING (-0.1604803 51.4352873, -0.1601615 ...",trunk,yes,30 mph,Upper Tooting Road,A24
3,2020-02-01 03:00:00,691007296,21390008,1425033102,34.161,21390008-1425033102,"LINESTRING (-0.1604803 51.4352873, -0.1601615 ...",trunk,yes,30 mph,Upper Tooting Road,A24
4,2020-02-01 04:00:00,691007296,21390008,1425033102,39.383,21390008-1425033102,"LINESTRING (-0.1604803 51.4352873, -0.1601615 ...",trunk,yes,30 mph,Upper Tooting Road,A24
...,...,...,...,...,...,...,...,...,...,...,...,...
493147,2020-02-22 19:00:00,24704286,130234354,1578999260,31.028,130234354-1578999260,"LINESTRING (0.0203705 51.5457204, 0.0211241 51...",primary,yes,30 mph,Romford Road,A118
493148,2020-02-22 20:00:00,24704286,130234354,1578999260,32.845,130234354-1578999260,"LINESTRING (0.0203705 51.5457204, 0.0211241 51...",primary,yes,30 mph,Romford Road,A118
493149,2020-02-22 21:00:00,24704286,130234354,1578999260,32.845,130234354-1578999260,"LINESTRING (0.0203705 51.5457204, 0.0211241 51...",primary,yes,30 mph,Romford Road,A118
493150,2020-02-22 22:00:00,24704286,130234354,1578999260,35.673,130234354-1578999260,"LINESTRING (0.0203705 51.5457204, 0.0211241 51...",primary,yes,30 mph,Romford Road,A118


In [9]:
train_data = train_data_w_info
print(train_data.isnull().sum())
print(train_data.duplicated().sum())

waktu_setempat      0
id_jalan            0
id_titik_mulai      0
id_titik_akhir      0
rerata_kecepatan    0
rute                0
geo                 0
highway             0
lit                 0
maxspeed            0
name                0
ref                 0
dtype: int64
0


In [10]:
train = train_data.rename(columns={
    'rute': 'unique_id', 
    'waktu_setempat': 'ds', 
    'rerata_kecepatan': 'y'
})
train.head()

,ds,id_jalan,id_titik_mulai,id_titik_akhir,y,unique_id,geo,highway,lit,maxspeed,name,ref
0,2020-02-01 00:00:00,691007296,21390008,1425033102,29.126,21390008-1425033102,"LINESTRING (-0.1604803 51.4352873, -0.1601615 ...",trunk,yes,30 mph,Upper Tooting Road,A24
1,2020-02-01 01:00:00,691007296,21390008,1425033102,29.126,21390008-1425033102,"LINESTRING (-0.1604803 51.4352873, -0.1601615 ...",trunk,yes,30 mph,Upper Tooting Road,A24
2,2020-02-01 02:00:00,691007296,21390008,1425033102,34.772,21390008-1425033102,"LINESTRING (-0.1604803 51.4352873, -0.1601615 ...",trunk,yes,30 mph,Upper Tooting Road,A24
3,2020-02-01 03:00:00,691007296,21390008,1425033102,34.161,21390008-1425033102,"LINESTRING (-0.1604803 51.4352873, -0.1601615 ...",trunk,yes,30 mph,Upper Tooting Road,A24
4,2020-02-01 04:00:00,691007296,21390008,1425033102,39.383,21390008-1425033102,"LINESTRING (-0.1604803 51.4352873, -0.1601615 ...",trunk,yes,30 mph,Upper Tooting Road,A24


In [11]:
train['maxspeed'] = train['maxspeed'].str.extract('(\d+)').astype(int)


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 493152 entries, 0 to 493151
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   ds              493152 non-null  datetime64[ns]
 1   id_jalan        493152 non-null  int64         
 2   id_titik_mulai  493152 non-null  int64         
 3   id_titik_akhir  493152 non-null  int64         
 4   y               493152 non-null  float64       
 5   unique_id       493152 non-null  object        
 6   geo             493152 non-null  object        
 7   highway         493152 non-null  object        
 8   lit             493152 non-null  object        
 9   maxspeed        493152 non-null  int64         
 10  name            493152 non-null  object        
 11  ref             493152 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(4), object(6)
memory usage: 48.9+ MB


In [32]:
train['maxspeed'] = str(train['maxspeed']).split(' ')[0]
train['maxspeed']

0         0
1         0
2         0
3         0
4         0
         ..
493147    0
493148    0
493149    0
493150    0
493151    0
Name: maxspeed, Length: 493152, dtype: object

In [13]:
#split data into train and valid

training = train[['ds','y','unique_id','highway','maxspeed']][train['ds'] < "2020-02-18"]
valid = train[['ds','y','unique_id','highway','maxspeed']][train['ds'] >= "2020-02-18"]
horizon = valid['ds'].nunique()

In [19]:
!pip install mlforecast
!pip install lightgbm
!pip install xgboost
!pip install window_ops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.1 MB/s eta 0:00:00


In [20]:
from mlforecast import MLForecast
from mlforecast.target_transforms import Differences
from mlforecast.utils import PredictionIntervals
from window_ops.expanding import expanding_mean
from window_ops.ewm import ewm_mean
from xgboost import XGBRegressor


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
SEED = 42

models = [XGBRegressor(max_depth=10, eval_metric='mape', random_state=SEED)]

In [7]:
mlf = MLForecast(
    models=models,  # List of models for forecasting: LightGBM, XGBoost and Linear Regression
    freq='H',  # Frequency of the data - 'D' for daily frequency
    differences=[24],
    lags=[24 * (i+1) for i in range(7)],  # Specific lags to use as regressors: 1 to 6 days
    lag_transforms = {
        24: [(ewm_mean, 0.3)],
    },
    date_features=['dayofweek', 'hour', 'week'],  # Date features to use as regressors
)

/opt/conda/lib/python3.10/site-packages/mlforecast/core.py:151: UserWarning: The differences argument is deprecated and will be removed in a future version.
Please pass an `mlforecast.target_transforms.Differences` instance to the `target_transforms` argument instead.
  warnings.warn(


In [14]:
def smape(actual, forecast):
    return 100 * np.mean(2 * np.abs(actual - forecast) / (np.abs(actual) + np.abs(forecast)))

In [10]:
import time

# Start the timer to calculate the time taken for fitting the models
start_time = time.time()

# Fit the MLForecast models to the data, with prediction intervals set using a window size of window_size days
mlf.fit(training, id_col='unique_id', time_col='ds', target_col='y', static_features=[])

# Calculate the end time after fitting the models
elapsed_time = time.time() - start_time

mins = int(elapsed_time // 60)
sec = int(elapsed_time % 60)

print(f'Total Training time : {mins} Mins {sec} Seconds')

preds = mlf.predict(horizon=horizon)
preds = preds.merge(valid[['ds','unique_id','y']], on=['ds','unique_id'], how='left')

error = smape(preds['y'],preds['XGBRegressor'])
print(f'SMAPE = {error}')

Total Training time : 0 Mins 54 Seconds


KeyError: 'LGBMRegressor'

In [11]:
error = smape(preds['y'],preds['XGBRegressor'])
print(f'SMAPE = {error}')

SMAPE = 9.196527957102846


# Tuning bersama Optuna sayangku

In [16]:
import time

In [51]:
import optuna

def objective(trial):
    #max_depth = trial.suggest_int('max_depth',10,50,step=5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-3, 1e-1)
    subsample = trial.suggest_uniform('subsample',0.1,1.0)
    colsample_bytree = trial.suggest_uniform('colsample_bytree', 0.1, 1.0)
    

    
    models = [XGBRegressor(max_depth=10,
                           eval_metric='mape',
                           random_state=42,
                           n_estimators=500,
                           learning_rate=learning_rate,
                           subsample=subsample,
                           colsample_bytree=colsample_bytree,
                          )]
    
    mlf = MLForecast(
    models=models,  # List of models for forecasting: LightGBM, XGBoost and Linear Regression
    freq='H',  # Frequency of the data - 'D' for daily frequency
    differences=[24],
    lags=[24 * (i+1) for i in range(7)],  # Specific lags to use as regressors: 1 to 6 days
    lag_transforms = {
        24: [(ewm_mean, 0.3)],
    },
    date_features=['dayofweek', 'hour', 'week'],  # Date features to use as regressors
)
    
    # Fit the MLForecast models to the data, with prediction intervals set using a window size of window_size days
    mlf.fit(training[['unique_id','y','ds','maxspeed']], id_col='unique_id', time_col='ds', target_col='y', static_features=['maxspeed'])

    preds = mlf.predict(horizon=horizon)
    preds = preds.merge(valid[['ds','unique_id','y']], on=['ds','unique_id'], how='left')

    error = smape(preds['y'],preds['XGBRegressor'])
    
    return error

In [52]:
# Start the timer to calculate the time taken for fitting the models
start_time = time.time()

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

# Calculate the end time after fitting the models
elapsed_time = time.time() - start_time

mins = int(elapsed_time // 60)
sec = int(elapsed_time % 60)

print(f'Total Tuning time : {mins} Mins {sec} Seconds')

[I 2023-08-09 14:18:22,010] A new study created in memory with name: no-name-400a2c59-3ee5-466d-8d43-6420be008e1a
/tmp/ipykernel_28/3663673830.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-3, 1e-1)
/tmp/ipykernel_28/3663673830.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  subsample = trial.suggest_uniform('subsample',0.1,1.0)
/tmp/ipykernel_28/3663673830.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  colsample_bytree = trial.suggest_uniform('colsample_bytree', 0.1, 1.0)


Total Tuning time : 83 Mins 50 Seconds


In [54]:
study.best_value

8.633048019038467

In [55]:
study.best_params

{'learning_rate': 0.01829164200773041,
 'subsample': 0.3021163442400033,
 'colsample_bytree': 0.7919662193666952}

# Training Akhir + Prediksi Akhir setelah Tuning

In [15]:
training_set_final = train[['ds','y','unique_id','highway','maxspeed']]
training_set_final

,ds,y,unique_id,highway,maxspeed
0,2020-02-01 00:00:00,29.126,21390008-1425033102,trunk,30
1,2020-02-01 01:00:00,29.126,21390008-1425033102,trunk,30
2,2020-02-01 02:00:00,34.772,21390008-1425033102,trunk,30
3,2020-02-01 03:00:00,34.161,21390008-1425033102,trunk,30
4,2020-02-01 04:00:00,39.383,21390008-1425033102,trunk,30
...,...,...,...,...,...
493147,2020-02-22 19:00:00,31.028,130234354-1578999260,primary,30
493148,2020-02-22 20:00:00,32.845,130234354-1578999260,primary,30
493149,2020-02-22 21:00:00,32.845,130234354-1578999260,primary,30
493150,2020-02-22 22:00:00,35.673,130234354-1578999260,primary,30


In [16]:
test_data

,id,waktu_setempat,id_jalan,id_titik_mulai,id_titik_akhir
0,0,2020-02-23 00:00:00+00:00,4004732,32046542,6454026544
1,1,2020-02-23 00:00:00+00:00,182210371,1314925464,1314925496
2,2,2020-02-23 00:00:00+00:00,22932408,1482086782,26481020
3,3,2020-02-23 00:00:00+00:00,182210371,3892883,267337489
4,4,2020-02-23 00:00:00+00:00,66924592,266041030,2592978110
...,...,...,...,...,...
127484,127484,2020-02-29 23:00:00+00:00,8504977,2378878,4712485405
127485,127485,2020-02-29 23:00:00+00:00,26785612,265111477,106201
127486,127486,2020-02-29 23:00:00+00:00,47010584,196038,1677146686
127487,127487,2020-02-29 23:00:00+00:00,22932408,873733547,1692210206


In [27]:
test_data['unique_id'] = test_data['id_titik_mulai'].astype(str) + "-" + test_data['id_titik_akhir'].astype(str)
test_data

,id,waktu_setempat,id_jalan,id_titik_mulai,id_titik_akhir,unique_id
0,0,2020-02-23 00:00:00+00:00,4004732,32046542,6454026544,32046542-6454026544
1,1,2020-02-23 00:00:00+00:00,182210371,1314925464,1314925496,1314925464-1314925496
2,2,2020-02-23 00:00:00+00:00,22932408,1482086782,26481020,1482086782-26481020
3,3,2020-02-23 00:00:00+00:00,182210371,3892883,267337489,3892883-267337489
4,4,2020-02-23 00:00:00+00:00,66924592,266041030,2592978110,266041030-2592978110
...,...,...,...,...,...,...
127484,127484,2020-02-29 23:00:00+00:00,8504977,2378878,4712485405,2378878-4712485405
127485,127485,2020-02-29 23:00:00+00:00,26785612,265111477,106201,265111477-106201
127486,127486,2020-02-29 23:00:00+00:00,47010584,196038,1677146686,196038-1677146686
127487,127487,2020-02-29 23:00:00+00:00,22932408,873733547,1692210206,873733547-1692210206


In [17]:
horizon = test_data['waktu_setempat'].nunique()
horizon

168

In [21]:
models = [XGBRegressor(max_depth=10,
                           eval_metric='mape',
                           random_state=42,
                           n_estimators=500,
                           learning_rate=0.01829164200773041,
                           subsample=0.3021163442400033,
                           colsample_bytree=0.7919662193666952,
                          )]
    
mlf = MLForecast(
    models=models,  # List of models for forecasting: LightGBM, XGBoost and Linear Regression
    freq='H',  # Frequency of the data - 'D' for daily frequency
    differences=[24],
    lags=[24 * (i+1) for i in range(7)],  # Specific lags to use as regressors: 1 to 6 days
    lag_transforms = {
        24: [(ewm_mean, 0.3)],
    },
    date_features=['dayofweek', 'hour', 'week'],  # Date features to use as regressors
)

/opt/conda/lib/python3.10/site-packages/mlforecast/core.py:151: UserWarning: The differences argument is deprecated and will be removed in a future version.
Please pass an `mlforecast.target_transforms.Differences` instance to the `target_transforms` argument instead.
  warnings.warn(


In [22]:
mlf.fit(training_set_final[['ds','y','unique_id','maxspeed']], id_col='unique_id', time_col='ds', target_col='y', static_features=['maxspeed'])
preds = mlf.predict(horizon=horizon)

In [23]:
preds

,unique_id,ds,XGBRegressor
0,101831-905115460,2020-02-23 00:00:00,31.758020
1,101831-905115460,2020-02-23 01:00:00,30.441917
2,101831-905115460,2020-02-23 02:00:00,31.603710
3,101831-905115460,2020-02-23 03:00:00,33.709698
4,101831-905115460,2020-02-23 04:00:00,33.609501
...,...,...,...
156907,997589816-997589768,2020-02-29 19:00:00,44.459278
156908,997589816-997589768,2020-02-29 20:00:00,44.650806
156909,997589816-997589768,2020-02-29 21:00:00,45.592445
156910,997589816-997589768,2020-02-29 22:00:00,45.500393


In [20]:
preds.duplicated().sum()

0

In [21]:
preds.isna().sum()

unique_id       0
ds              0
XGBRegressor    0
dtype: int64

In [22]:
preds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156912 entries, 0 to 156911
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   unique_id     156912 non-null  object        
 1   ds            156912 non-null  datetime64[ns]
 2   XGBRegressor  156912 non-null  float32       
dtypes: datetime64[ns](1), float32(1), object(1)
memory usage: 3.0+ MB


In [24]:
preds = preds.rename(columns={'ds':'waktu_setempat','XGBRegressor':'rerata_kecepatan'})
preds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156912 entries, 0 to 156911
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   unique_id         156912 non-null  object        
 1   waktu_setempat    156912 non-null  datetime64[ns]
 2   rerata_kecepatan  156912 non-null  float32       
dtypes: datetime64[ns](1), float32(1), object(1)
memory usage: 3.0+ MB


In [25]:
preds['waktu_setempat'] = preds['waktu_setempat']dt.tz_localize('UTC')
preds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156912 entries, 0 to 156911
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype              
---  ------            --------------   -----              
 0   unique_id         156912 non-null  object             
 1   waktu_setempat    156912 non-null  datetime64[ns, UTC]
 2   rerata_kecepatan  156912 non-null  float32            
dtypes: datetime64[ns, UTC](1), float32(1), object(1)
memory usage: 3.0+ MB


In [26]:
preds.head()

,unique_id,waktu_setempat,rerata_kecepatan
0,101831-905115460,2020-02-23 00:00:00+00:00,31.758020
1,101831-905115460,2020-02-23 01:00:00+00:00,30.441917
2,101831-905115460,2020-02-23 02:00:00+00:00,31.603710
3,101831-905115460,2020-02-23 03:00:00+00:00,33.709698
4,101831-905115460,2020-02-23 04:00:00+00:00,33.609501


In [28]:
preds.to_csv('predictions.csv')

In [29]:
submission = test_data.merge(preds[['waktu_setempat','unique_id','rerata_kecepatan']],on=['waktu_setempat','unique_id'],how='left')
submission

,id,waktu_setempat,id_jalan,id_titik_mulai,id_titik_akhir,unique_id,rerata_kecepatan
0,0,2020-02-23 00:00:00+00:00,4004732,32046542,6454026544,32046542-6454026544,42.136974
1,1,2020-02-23 00:00:00+00:00,182210371,1314925464,1314925496,1314925464-1314925496,41.264324
2,2,2020-02-23 00:00:00+00:00,22932408,1482086782,26481020,1482086782-26481020,36.632042
3,3,2020-02-23 00:00:00+00:00,182210371,3892883,267337489,3892883-267337489,42.693665
4,4,2020-02-23 00:00:00+00:00,66924592,266041030,2592978110,266041030-2592978110,31.211496
...,...,...,...,...,...,...,...
127484,127484,2020-02-29 23:00:00+00:00,8504977,2378878,4712485405,2378878-4712485405,30.979939
127485,127485,2020-02-29 23:00:00+00:00,26785612,265111477,106201,265111477-106201,37.550995
127486,127486,2020-02-29 23:00:00+00:00,47010584,196038,1677146686,196038-1677146686,40.180294
127487,127487,2020-02-29 23:00:00+00:00,22932408,873733547,1692210206,873733547-1692210206,35.397579


In [37]:
submission_file = submission[['id','rerata_kecepatan']]
submission_file.to_csv('submission.csv',index=False)

In [31]:
submission_file[]

,id,rerata_kecepatan
0,0,42.136974
1,1,41.264324
2,2,36.632042
3,3,42.693665
4,4,31.211496
...,...,...
127484,127484,30.979939
127485,127485,37.550995
127486,127486,40.180294
127487,127487,35.397579


In [32]:
test_data

,id,waktu_setempat,id_jalan,id_titik_mulai,id_titik_akhir,unique_id
0,0,2020-02-23 00:00:00+00:00,4004732,32046542,6454026544,32046542-6454026544
1,1,2020-02-23 00:00:00+00:00,182210371,1314925464,1314925496,1314925464-1314925496
2,2,2020-02-23 00:00:00+00:00,22932408,1482086782,26481020,1482086782-26481020
3,3,2020-02-23 00:00:00+00:00,182210371,3892883,267337489,3892883-267337489
4,4,2020-02-23 00:00:00+00:00,66924592,266041030,2592978110,266041030-2592978110
...,...,...,...,...,...,...
127484,127484,2020-02-29 23:00:00+00:00,8504977,2378878,4712485405,2378878-4712485405
127485,127485,2020-02-29 23:00:00+00:00,26785612,265111477,106201,265111477-106201
127486,127486,2020-02-29 23:00:00+00:00,47010584,196038,1677146686,196038-1677146686
127487,127487,2020-02-29 23:00:00+00:00,22932408,873733547,1692210206,873733547-1692210206


In [34]:
print(submission_file.isnull().sum())
print(submission_file.duplicated().sum())

id                  0
rerata_kecepatan    0
dtype: int64
0


In [35]:
submission_file

,id,rerata_kecepatan
0,0,42.136974
1,1,41.264324
2,2,36.632042
3,3,42.693665
4,4,31.211496
...,...,...
127484,127484,30.979939
127485,127485,37.550995
127486,127486,40.180294
127487,127487,35.397579
